# Making Pandas DataFrames from API Requests
In this example, we will use the U.S. Geological Survey's API to grab a JSON of earthquake data and convert it to a pandas DataFrame.

USGS API: https://earthquake.usgs.gov/fdsnws/event/1/

### Get Data from API

In [1]:
import requests
import datetime
import pandas as pd

yesterday = datetime.date.today() - datetime.timedelta(1)
api = 'https://earthquake.usgs.gov/fdsnws/event/1/query'
payload = {
    'format' : 'geojson',
    'starttime' : yesterday - datetime.timedelta(26),
    'endtime' : yesterday
}
response = requests.get(api, params=payload)

# let's make sure the request was OK
print(response.status_code)

200


Response of 200 means OK, so we can pull the data out of the result. Since we asked the API for a JSON, we can extract it from the response with the JSON method.

### Isolate the Data from the JSON Response
We need to check the structures of the response data to know where our data is.

In [2]:
earthquake_json = response.json()
earthquake_json.keys()

dict_keys(['type', 'metadata', 'features', 'bbox'])

In [3]:
earthquake_json['metadata']

{'generated': 1583749181000,
 'url': 'https://earthquake.usgs.gov/fdsnws/event/1/query?format=geojson&starttime=2020-02-11&endtime=2020-03-08',
 'title': 'USGS Earthquakes',
 'status': 200,
 'api': '1.8.1',
 'count': 10009}

Each element in the JSON array `features` is a row of data for our dataframe.

In [4]:
type(earthquake_json['features'])

list

Your data will be different depending on the date you run this.

In [9]:
earthquake_json['features'][0]

{'type': 'Feature',
 'properties': {'mag': 1.75,
  'place': '11km NE of Coso Junction, CA',
  'time': 1583625168950,
  'updated': 1583625829990,
  'tz': -480,
  'url': 'https://earthquake.usgs.gov/earthquakes/eventpage/ci38386994',
  'detail': 'https://earthquake.usgs.gov/fdsnws/event/1/query?eventid=ci38386994&format=geojson',
  'felt': None,
  'cdi': None,
  'mmi': None,
  'alert': None,
  'status': 'automatic',
  'tsunami': 0,
  'sig': 47,
  'net': 'ci',
  'code': '38386994',
  'ids': ',ci38386994,',
  'sources': ',ci,',
  'types': ',focal-mechanism,geoserve,nearby-cities,origin,phase-data,scitech-link,',
  'nst': 20,
  'dmin': 0.006389,
  'rms': 0.13,
  'gap': 100,
  'magType': 'ml',
  'type': 'earthquake',
  'title': 'M 1.8 - 11km NE of Coso Junction, CA'},
 'geometry': {'type': 'Point', 'coordinates': [-117.8573333, 36.1115, 2.12]},
 'id': 'ci38386994'}

### Convert to DataFrame
We need to grab the `properties` section out of every entry in the `features` JSON array to create our dataframe.

In [10]:
earthquake_properties_data = [data['properties'] for data in earthquake_json['features']]
df = pd.DataFrame(earthquake_properties_data)
df.head()

,alert,cdi,code,detail,dmin,felt,gap,ids,mag,magType,...,sources,status,time,title,tsunami,type,types,tz,updated,url
0,None,NaN,38386994,https://earthquake.usgs.gov/fdsnws/event/1/que...,0.006389,NaN,100.0,",ci38386994,",1.75,ml,...,",ci,",automatic,1583625168950,"M 1.8 - 11km NE of Coso Junction, CA",0,earthquake,",focal-mechanism,geoserve,nearby-cities,origin...",-480,1583625829990,https://earthquake.usgs.gov/earthquakes/eventp...
1,None,2.2,2020067037,https://earthquake.usgs.gov/fdsnws/event/1/que...,0.134800,3.0,255.0,",pr2020067037,",2.96,md,...,",pr,",reviewed,1583624685130,"M 3.0 - 14km SSE of Guanica, Puerto Rico",0,earthquake,",dyfi,geoserve,origin,phase-data,",-240,1583627751478,https://earthquake.usgs.gov/earthquakes/eventp...
2,None,NaN,020336wdty,https://earthquake.usgs.gov/fdsnws/event/1/que...,NaN,NaN,NaN,",ak020336wdty,",1.50,ml,...,",ak,",automatic,1583624021565,"M 1.5 - 23km ENE of Cape Yakataga, Alaska",0,earthquake,",geoserve,origin,",-540,1583624232043,https://earthquake.usgs.gov/earthquakes/eventp...
3,None,NaN,020336wamw,https://earthquake.usgs.gov/fdsnws/event/1/que...,NaN,NaN,NaN,",ak020336wamw,",1.20,ml,...,",ak,",reviewed,1583623977854,"M 1.2 - 23km ENE of Cape Yakataga, Alaska",0,earthquake,",geoserve,origin,phase-data,",-540,1583626078329,https://earthquake.usgs.gov/earthquakes/eventp...
4,None,NaN,38386978,https://earthquake.usgs.gov/fdsnws/event/1/que...,0.208500,NaN,187.0,",ci38386978,",1.68,ml,...,",ci,",automatic,1583623636370,"M 1.7 - 18km SSE of Gorman, CA",0,earthquake,",geoserve,nearby-cities,origin,phase-data,scit...",-480,1583623879658,https://earthquake.usgs.gov/earthquakes/eventp...


### (Optional) Write Data to CSV

In [11]:
pd.DataFrame(earthquake_properties_data).to_csv('earthquakes.csv', index=False)